# 1. 简介
训练模型时需要使用可视化工具监控训练情况，不对劲时就要及时停止。

TensorBoard 是 TensorFlow 中强大的可视化工具，支持标量、文本、图像、音频、视频和 Embedding 等多种数据可视化，在 PyTorch 中也可以使用 TensorBoard。使用之前，需要安装 TensorFlow 和 TensorBoard，两者版本需要兼容。

**TensorBoard 可视化的流程**：
- **首先编写 Python 代码把需要可视化的数据以 event file 的形式存储到硬盘中**：保存 event file 需要用到 SummaryWriter 类，这个类是用于保存数据的最重要的类，执行完后，会在当前文件夹生成一个 runs 的文件夹，里面保存的就是数据的 event file。
- **然后在终端（Terminal）使用 TensorBoard 这个工具从硬盘中读取 event file**：在 Terminal 中先 cd 进入相应的文件夹，然后输入 `tensorboard --logdir=./runs` 启动 tensorboard 服务，./runs 是 runs 文件夹的路径。
- **然后 TensorBoard 在网页端将数据可视化**：命令行会显示 tensorboard 的访问地址 `TensorBoard 2.16.2 at http://localhost:6006/ (Press CTRL+C to quit)`，点击网址，在浏览器中打开。

**网页端简要介绍：**
- 设置：可以设置刷新获取数据的时间间隔，在模型训练时可以实时监控数据的变化。
- Show data download links：可以展示每个图的下载按钮，选中需要下载的曲线，然后下载，格式有 csv 和 json 可选。
- Ignore outliers in chart scaling：可以设置是否忽略离群点
- Soothing 是对图像进行平滑，颜色较淡的阴影部分才是真正的曲线数据，Smoothing 设置为 0，没有进行平滑。
- Horizontal Axis表示横轴：STEP 表示原始数据作为横轴，RELATIVE 和 WALL 都是以时间作为横轴，RELATIVE 是相对时间，单位是小时，WALL 是绝对时间。
- runs：显示所有的 event file，可以选择展示某些 event file 的图像，其中正方形按钮是多选，圆形按钮是单选。
- Filter tags：可以根据 tags 来搜索图像

# 2. SummaryWriter 类

`torch.utils.tensorboard.SummaryWriter(log_dir=None, comment='', purge_step=None, max_queue=10, flush_secs=120, filename_suffix='')`

**功能**：提供创建 event file 的高级接口

**主要属性：**
- log_dir：event file 输出文件夹，不设置的话，默认为当前文件夹
- comment：不设置 log_dir 时，runs 文件夹里的子文件夹会以 comment 内容为后缀；设置时不会
- filename_suffix：event file 文件名后缀

**主要方法：**

## 2.1 add_scalar()
```
writer = SummaryWriter()
writer.add_scalar(tag, scalar_value, global_step=None, walltime=None)
```
**功能**：记录标量

**主要参数**：
- tag：图像的标签名，图的唯一标识
- scalar_value：要记录的标量（loss、accuracy 等），y 轴的数据
- global_step：为 for i in range(n) 里面的 i，一般为 epoch 或 iter_count，是以 x 轴为序显示，记录这是第几个子图

**注意**：输出的 loss 等一般为张量，scalar_value 需要是标量，所以需要经过 `loss.item()` 将 loss 转换为标量。 

## 2.2 add_scalars()

上面的 add_scalar() 只能记录一条曲线的数据，但是我们在实际中可能需要在一张图中同时展示多条曲线，这时我们可以使用 add_scalars() 方法。

```
writer = SummaryWriter()
writer.add_scalars(main_tag, tag_scalar_dict, global_step=None, walltime=None)
```
**功能**：记录标量

**主要参数**：
- main_tag：该图的标签
- tag_scalar_dict：用字典的形式记录多个曲线，key 是要记录的标量的 tag，value 是要记录的标量的值

**应用：**
```
import numpy as np
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(comment='test_tensorboard')

for x in range(100):
    writer.add_scalar('y=2x', x * 2, x)
    writer.add_scalar('y=pow(2, x)', 2 ** x, x)

    writer.add_scalars('data/scalar_group', {"xsinx": x * np.sin(x),
                                             "xcosx": x * np.cos(x),
                                             "arctanx": np.arctan(x)}, x)

writer.close()
```

## 2.3 add_histogram()
```
writer = SummaryWriter()
writer.add_histogram(tag, values, global_step=None, bins='tensorflow', walltime=None, max_bins=None)
```
**功能**：统计参数的直方图与多分位数折线图

**主要参数**：
- tag：图像的标签名，图的唯一标识
- values：要统计的参数（权值、偏置或者梯度等）
- global_step：为 for i in range(n) 里面的 i，一般为 epoch 或 iter_count，是以 y 轴为序显示，记录这是第几个子图
- bins：取直方图的 bins，使用默认值即可

除此之外，还会得到 DISTRIBUTIONS，这是多分位折线图，纵轴有 9 个折线，表示数据的分布区间，某个区间的颜色越深，表示这个区间的数所占比例越大。横轴是 global_step。这个图的作用是观察参数的分布和方差。

**注意**：
- 如果参数的梯度都是接近于 0，说明模型很快就收敛了。
- 通常我们使用这种方法查看我们的网络参数在训练时的分布变化情况，如果分布很奇怪，并且 loss 没有下降，这时需要考虑是什么原因。
- 如果前面网络层的梯度很小，后面网络层的梯度比较大，那么可能是梯度消失，因为后面网络层的较大梯度反向传播到前面网络层时已经变小了。
- 如果前后网络层的梯度都很小，那么说明不是梯度消失，而是因为 loss 很小，模型已经接近收敛。

**应用：**
```
# 在 epoch 循环下，记录在每个 epoch 中，每个网络层的参数（权值）和参数（权值）的梯度
for epoch in range(MAX_EPOCH):

    ···

    for layer_name, layer_param in net.named_parameters():
        writer.add_histogram(layer_name + '_grad', layer_param.grad, epoch)  # 记录参数的梯度
        writer.add_histogram(layer_name + '_data', layer_param.data, epoch)  # 记录参数
```

`net.named_parameters()` 会得到模型每一层的名字和每一层的参数

由于 `nn.Parameter` 是一个张量子类，所以 layer_param 是一个张量，可以使用 `.grad` 获取张量的梯度属性，使用 `.data` 获取张量的 data 属性。

## 2.4 add_image()
```
writer = SummaryWriter()
writer.add_image(tag, img_tensor, global_step=None, walltime=None, dataformats='CHW')
```
**功能**：记录图像，可视化图像，可视化卷积层

**主要参数**：
- tag：图像的标签名，图像的唯一标识
- img_tensor：图像数据，需要注意尺度。如果像素值在 `[0, 1]` 之间，那么默认会乘以 255，放大到 `[0, 255]` 范围之间；如果有大于 1 的像素值，认为已经是 `[0, 255]` 范围，那么就不会放大。
- global_step：为 for i in range(n) 里面的 i，一般为 epoch 或 iter_count，是以 x 轴为序显示，记录这是第几个子图
- dataformats：数据形式，取值有'CHW'，'HWC'，'HW'。

<br/>

`add_image()` 记录的图像会根据 step(global_step) 一张一张图地显示，不是很方便，下面介绍在一个网格中显示很多图片的方法：
```
torchvision.utils.make_grid(tensor, nrow=8, padding=2, normalize=False, range=None, scale_each=False, pad_value=0)
```
**功能**：制作网格图像

**主要参数**：
- tensor：需要记录可视化的图像数据，$B \times C \times H \times W$ 的形状
- nrow：行数(列数是自动计算的，为：$\frac{B}{nrow}$)
- padding：图像间距，单位是像素，默认为 2。
- normalize：是否将像素值标准化到 `[0, 255]` 之间
- range：标准化范围，例如原图的像素值范围是 `[-1000, 2000]`，设置 range 为 `[-600, 500]`，那么会把小于 -600 的像素值变为 -600，那么会把大于 500 的像素值变为 500，然后标准化到 `[0, 255]` 之间。
- scale_each：是否单张图维度标准化
- pad_value：图像间隔的像素值，默认为 0，为黑色边。

## 2.5 add_graph()
```
writer = SummaryWriter()
writer.add_graph(model, input_to_model=None, verbose=False)
```
**功能**：可视化模型计算图

**主要参数**：
- model：继承自 nn.Module 的 pytorch 模型
- input_to_model：输入给模型的数据，形状为 BCHW，必须是四维的
- verbose：是否打印图结构信息

**查看 LeNet 的计算图代码如下**：
```
writer = SummaryWriter(comment='test_your_comment', filename_suffix="_test_your_filename_suffix")
fake_img = torch.randn(1, 3, 32, 32)
lenet = LeNet(classes=2)
writer.add_graph(lenet, fake_img)
writer.close()
```